In [1]:
import os

In [2]:
%pwd

'c:\\Users\\bisht\\OneDrive\\Desktop\\Food Recommedor\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\bisht\\OneDrive\\Desktop\\Food Recommedor'

In [6]:
import tensorflow as tf

In [7]:
model=tf.keras.models.load_model('artifacts/prepare_base_model/base_model_updated.h5')

In [8]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model:Path
    training_data:Path
    extract_feature:Path
    csv_file:Path
    mlflow_uri:str
     

In [9]:
from src.Food_Recommendation.constants import *
from src.Food_Recommendation.utils.common import read_yaml,create_directories,save_json

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    def get_components(self)->EvaluationConfig:
        components=EvaluationConfig(
            path_of_model='artifacts/prepare_base_model/base_model_updated.h5',
            training_data='artifacts/data_ingestion/data/data',
            mlflow_uri='https://dagshub.com/AbhayBisht0801/Food-Recommendation-System.mlflow',
            extract_feature='artifacts/training/extracted_feature.pkl',
            csv_file='artifacts/data_ingestion/cuisine_update.csv'
        )
        return components
    


In [11]:
import os
import tensorflow as tf
import time
from urllib.parse import urlparse
from zipfile import ZipFile
import mlflow
import mlflow.keras

In [12]:
class Evaluation:
    def __init__(self,config:EvaluationConfig):
        self.config=config
        def load_model(path: Path) -> tf.keras.Model:
            return tf.keras.models.load_model(path)
        self.model=load_model(self.config.path_of_model)
    def log_into_mlflow(self):  
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
             mlflow.log_artifact("artifacts/training/extracted_feature.pkl", artifact_path="extracted_features")
             mlflow.log_artifact("artifacts/data_ingestion/cuisine_updated.csv", artifact_path="cusine_data")
            
            # Model registry does not work with file store
             if tracking_url_type_store != "file":
                 mlflow.keras.log_model(self.model, "model", registered_model_name="Feature_extractor")
             else:
                 mlflow.keras.log_model(self.model, "model")
    

In [13]:
try:
    config=ConfigurationManager()
    eval_config=config.get_components()
    evaluation=Evaluation(eval_config)
    evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2024-01-27 10:10:14,808: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-27 10:10:14,817: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-27 10:10:14,819: INFO: common: created directory at: artifacts]


[2024-01-27 10:10:16,607: WARNING: hdf5_format: No training configuration found in the save file, so the model was *not* compiled. Compile it manually.]


2024/01/27 10:10:52 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-01-27 10:10:52,320: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
[2024-01-27 10:11:03,873: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 53). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\bisht\AppData\Local\Temp\tmpqmg135ob\model\data\model\assets
[2024-01-27 10:11:07,335: INFO: builder_impl: Assets written to: C:\Users\bisht\AppData\Local\Temp\tmpqmg135ob\model\data\model\assets]


c:\Users\bisht\OneDrive\Desktop\Food Recommedor\venv\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'Feature_extractor' already exists. Creating a new version of this model...
2024/01/27 10:12:15 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: Feature_extractor, version 6
Created version '6' of model 'Feature_extractor'.
